## LCEL
The output of the previous runnable's .invoke() call is passed as input to the next runnable.
like:
`chain = prompt | model | StrOutputParser()`

In [1]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
prompt.invoke({"topic":"Dog"})

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about Dog')])

We can even combine this chain with more runnables to create another chain. 
`analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")`
`composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()`

lecl 其实也可以这样写
```python
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)
```
或者这样
```python
composed_chain_with_pipe = RunnableParallel({"joke": chain}).pipe(
    analysis_prompt, model, StrOutputParser()
)
```



astream 是异步的打字机，等待完成后才输出
stream 是同步打字机

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=api_key)
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, flush=True)


The
 color
 of
 the
 sky
 can
 vary
 depending
 on
 the
 time
 of
 day
 and
 weather
 conditions
.
 During
 the
 day
,
 the
 sky
 is
 typically
 blue
,
 but
 it
 can
 also
 appear
 pink
,
 orange
,
 or
 purple
 during
 sunrise
 and
 sunset
.
 At
 night
,
 the
 sky
 appears
 dark
 blue
 or
 black
 with
 stars
 and
 sometimes
 the
 moon
 visible
.



: 

## 保存向量数据到磁盘
```python
vectorstore = Chroma.from_texts(texts, embedding=embeddings)
save_path = "path_to_save_vectorstore"
# 保存 vectorstore 到磁盘
vectorstore.save(save_path)

# # 重新加载 vectorstore

try:
    vectorstore = Chroma.load(save_path)
    print("Loaded vectorstore from disk.")
except FileNotFoundError:
    print("No saved vectorstore found. Creating a new one.")
    texts = ["harrison worked at kensho", "harrison likes spicy food"]
    embeddings = OpenAIEmbeddings(api_key=api_key)
    vectorstore = Chroma.from_texts(texts, embedding=embeddings)
    vectorstore.save(save_path)
    print("Saved new vectorstore to disk.")

# 创建 retriever
retriever = vectorstore.as_retriever()

# 使用 retriever
response = retriever.invoke('where did harrison work?')
```


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(api_key=api_key),
)
retriever = vectorstore.as_retriever()

retriever.invoke('where did harrison work?')

In [11]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableLambda

def reverse_word(word: str):
    return word[::-1]

reverse_word = RunnableLambda(reverse_word)

# @tool
# def bad_tool(word: str):
#     """Custom tool that doesn't propagate callbacks."""
#     return reverse_word.invoke(word)


# async for event in bad_tool.astream_events("hello", version="v2"):
#     print(event)


@tool
def correct_tool(word: str, callbacks):
    """A tool that correctly propagates callbacks."""
    return reverse_word.invoke(word, {"callbacks": callbacks})


async for event in correct_tool.astream_events("hello", version="v2"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'bad_tool', 'tags': [], 'run_id': 'ca771aad-2e2c-4c9a-886b-2620a4d4a467', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': '172b12e2-5c8f-48e9-b037-aff7bd0a0101', 'metadata': {}, 'parent_ids': ['ca771aad-2e2c-4c9a-886b-2620a4d4a467']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': '172b12e2-5c8f-48e9-b037-aff7bd0a0101', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['ca771aad-2e2c-4c9a-886b-2620a4d4a467']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': 'ca771aad-2e2c-4c9a-886b-2620a4d4a467', 'name': 'bad_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}
{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'correct_tool', 'tags': [], 'run_id': '4fca6351-b46b-40b1-b04c-209ea78a0ee0', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'i

In [12]:
from langchain_core.runnables import chain


@chain
async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2

await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234", version="v2"):
    print(event)



{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '3dd63972-4fea-4096-8313-d8bf92ccfa8a', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': '88f70cf8-cc6e-466d-917c-183e056b3734', 'metadata': {}, 'parent_ids': ['3dd63972-4fea-4096-8313-d8bf92ccfa8a']}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': '88f70cf8-cc6e-466d-917c-183e056b3734', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['3dd63972-4fea-4096-8313-d8bf92ccfa8a']}
{'event': 'on_chain_stream', 'run_id': '3dd63972-4fea-4096-8313-d8bf92ccfa8a', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '3dd63972-4fea-4096-8313-d8bf92ccfa8a', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}
